<a href="https://colab.research.google.com/github/minnnjecho/KUBIG_2020_FALL/blob/master/KU-BIG/2021_Spring/ch08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

T, H = 5, 4    # 시계열의 길이: T, 은닉 상태 벡터의 원소 수: H
hs = np.random.randn(T, H)
print(hs)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02]) # shape: (5, ), 가중치 벡터

ar = a.reshape(5, 1).repeat(4, axis=1) # hs와의 연산을 위해 확장(실제로는 안해줘도 됨)
print(ar)
print(ar.shape) # shape: (5, 4)

t = hs * ar # Encoder에서 받은 hs에 가중합(원소별 곱을 계산)
print(t)
print(t.shape)

c = np.sum(t, axis=0) # 0번째 축이 사라짐.
print(c.shape)

[[ 0.28402538  2.05846921 -0.24260861  0.76716   ]
 [-0.63235436  0.74719498  1.29141102 -1.02398598]
 [ 1.90489048 -1.34088943  1.47110725  1.49042319]
 [-0.53821012  0.46875563  0.76289666 -1.47590461]
 [ 0.68239529  0.1576866  -1.6650861  -0.44989876]]
[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]
(5, 4)
[[ 0.2272203   1.64677537 -0.19408689  0.613728  ]
 [-0.06323544  0.0747195   0.1291411  -0.1023986 ]
 [ 0.05714671 -0.04022668  0.04413322  0.0447127 ]
 [-0.02691051  0.02343778  0.03814483 -0.07379523]
 [ 0.01364791  0.00315373 -0.03330172 -0.00899798]]
(5, 4)
(4,)


In [ ]:
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)
# ar = a.reshape(N, T, 1) # 브로드캐스트를 사용하는 경우

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1)
print(c.shape)

(10, 5, 4)
(10, 4)


In [ ]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
    
    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape

        dt = dc.reshape(N, 1, H).repeat(T, axis=1) # sum의 역전파
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2) # repeat의 역전파

        return dhs, da

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/KU-BIG_2021-1_NLP_Study/input')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)
# hr = h.reshape(N, 1, H) # 브로드캐스트를 사용하는 경우

t = hs * hr
print(t.shape) # (10, 5, 4)

s = np.sum(t, axis=2)
print(s.shape)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape) # (10, 5)

(10, 5, 4)
(10, 5)
(10, 5)


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/KU-BIG_2021-1_NLP_Study/input')
from common.np import *
from common.layers import Softmax

class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = none
    
    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H).repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh

In [ ]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)     # AttentionWeight()의 backward
        dhs1, dh = self.attention_weight_layer.backward(da) # WeightSum()의 backward
        dhs = dhs0 + dhs1
        return dhs, dh

In [ ]:
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:, t, :])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)
        
        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :]) 
            dhs_enc += dhs
            dhs_dec[:, t, :] = h
        
        return dhs_enc, dhs_dec

## 어텐션을 갖춘 seq2seq 구현

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/KU-BIG_2021-1_NLP_Study/input')
from common.time_layers import *
from ch07.seq2seq import Encoder, Seq2seq
from ch08.attention_layer import TimeAttention

class AttentionEncoder(Encoder):
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        return hs
    
    def backward(self, dhs):
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

In [ ]:
class AttentionDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(2 * H, V) / np.sqrt(2 * H)).astype('f')
        affine_b = np.zeros(V).astype('f')


        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.attention = TimeAttnetion()
        self.affine = TimeAffine(affine_W, affine_b)
        layers = [self.embed, self.lstm, self.attention, self.affine]


        self.params, self.grads = [], []
        for layer in layers:
            self.params += layers.params
            self.grads += layer.grads
    

    def forward(self, xs, enc_hs):
        h = enc_hs[:, -1]
        self.lstm.set_state(h)


        out = self.embed.forward(xs)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)

        return score
    
    # def backward(self, dscore):
    #     # 깃허브의 소스 코드 참고
        
    # def generate(self, enc_hs, start_id, sample_size):
    #     # 깃허브의 소스 코드 참고

In [ ]:
from ch07.seq2seq import Encoder, Seq2seq

class AttentionSeq2seq(Seq2seq):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        args = vocab_size, wordvec_size, hidden_size
        self.encoder = AttentionEncoder(*args)
        self.decoder = AttentionDecoder(*args)
        self.softmax = TimeSoftmaxWithLoss()


        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/KU-BIG_2021-1_NLP_Study/input')
sys.path.append('/content/drive/Shareddrives/KU-BIG_2021-1_NLP_Study/input/ch07')
import numpy as np
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch08.attention_seq2seq import AttentionSeq2seq
from ch07.seq2seq import Seq2seq
from ch07.peeky_seq2seq import PeekySeq2seq

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 문장 반전
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

# 하이퍼파라미터 설절
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5

model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad)
    
    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse=True)
    
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' %(acc*100))

model.save_params()

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.08
| 에폭 1 |  반복 21 / 351 | 시간 11[s] | 손실 3.09
| 에폭 1 |  반복 41 / 351 | 시간 23[s] | 손실 1.90
| 에폭 1 |  반복 61 / 351 | 시간 35[s] | 손실 1.72
| 에폭 1 |  반복 81 / 351 | 시간 47[s] | 손실 1.46
| 에폭 1 |  반복 101 / 351 | 시간 59[s] | 손실 1.19
| 에폭 1 |  반복 121 / 351 | 시간 71[s] | 손실 1.14
| 에폭 1 |  반복 141 / 351 | 시간 84[s] | 손실 1.09
| 에폭 1 |  반복 161 / 351 | 시간 96[s] | 손실 1.06
| 에폭 1 |  반복 181 / 351 | 시간 108[s] | 손실 1.04
| 에폭 1 |  반복 201 / 351 | 시간 120[s] | 손실 1.03
| 에폭 1 |  반복 221 / 351 | 시간 132[s] | 손실 1.02
| 에폭 1 |  반복 241 / 351 | 시간 144[s] | 손실 1.02
| 에폭 1 |  반복 261 / 351 | 시간 156[s] | 손실 1.01
| 에폭 1 |  반복 281 / 351 | 시간 168[s] | 손실 1.00
| 에폭 1 |  반복 301 / 351 | 시간 181[s] | 손실 1.00
| 에폭 1 |  반복 321 / 351 | 시간 193[s] | 손실 1.00
| 에폭 1 |  반복 341 / 351 | 시간 205[s] | 손실 1.00
Q 10/15/94                     
T 1994-10-15
☒ 1978-08-11
---
Q thursday, november 13, 2008  
T 2008-11-13
☒ 1978-08-11
---
Q Mar 25, 2003                 
T 2003-03-25
☒ 1978-08-11
---
Q Tuesday, November 